In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

In [2]:
data = pd.read_csv("../data/raw/ai4i2020.csv")

In [3]:
# 1. 결측치 확인
print("결측치 확인:\n", data.isnull().sum())

결측치 확인:
 UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Tool wear [min]            0
Machine failure            0
TWF                        0
HDF                        0
PWF                        0
OSF                        0
RNF                        0
dtype: int64


In [4]:
# 3. 이상치 처리 (간단히 z-score 기반 예시)
from scipy.stats import zscore
numeric_cols = ["Air temperature [K]", "Process temperature [K]", "Rotational speed [rpm]", 
                "Torque [Nm]", "Tool wear [min]"]

data[numeric_cols] = data[numeric_cols].apply(lambda x: x[(zscore(x).abs() < 3)])

In [5]:
# 4. 범주형 변수 원-핫 인코딩
categorical_cols = ["Type"]
data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

In [6]:
# 5. Feature / Target 분리
X = data.drop(columns=["UDI", "Product ID", "Machine failure", "TWF","HDF","PWF","OSF","RNF"])
y = data["Machine failure"]

In [7]:
# 6. 학습/검증/테스트 분리
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp)

In [8]:
# 7. 정규화 (StandardScaler)
scaler = StandardScaler()
X_train[numeric_cols] = scaler.fit_transform(X_train[numeric_cols])
X_val[numeric_cols] = scaler.transform(X_val[numeric_cols])
X_test[numeric_cols] = scaler.transform(X_test[numeric_cols])

In [9]:
# 8. 데이터 확인
print("학습용 데이터 shape:", X_train.shape)
print("검증용 데이터 shape:", X_val.shape)
print("테스트용 데이터 shape:", X_test.shape)

학습용 데이터 shape: (7000, 7)
검증용 데이터 shape: (1500, 7)
테스트용 데이터 shape: (1500, 7)


In [11]:
# 9. 필요 시 CSV 저장
X_train.to_csv("../data/X_train.csv", index=False)
X_val.to_csv("../data/X_val.csv", index=False)
X_test.to_csv("../data/X_test.csv", index=False)
y_train.to_csv("../data/y_train.csv", index=False)
y_val.to_csv("../data/y_val.csv", index=False)
y_test.to_csv("../data/y_test.csv", index=False)